In [18]:
import os
import numpy as np
import cv2
from matplotlib import pyplot as plt
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input, decode_predictions

#from tensorflow.keras import backend as K
from keras import backend as K

import tensorflow as tf
from tensorflow.python.framework import ops

## Define model here

In [19]:
def build_model():
    """Function returning keras model instance.
    
    Model can be
     - Trained here
     - Loaded with load_model
     - Loaded from keras.applications
    """
    return VGG16(include_top=True, weights='imagenet')

H, W = 224, 224 # Input shape, defined by the model (model.input_shape)

### Utility functions

In [20]:
def load_image(path, preprocess=True):
    """Load and preprocess image."""
    x = image.load_img(path, target_size=(H, W))
    if preprocess:
        x = image.img_to_array(x)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)
    return x


def deprocess_image(x):
    """Same normalization as in:
    https://github.com/fchollet/keras/blob/master/examples/conv_filter_visualization.py
    """
    x = x.copy()
    if np.ndim(x) > 3:
        x = np.squeeze(x)
    # normalize tensor: center on 0., ensure std is 0.1
    x -= x.mean()
    x /= (x.std() + 1e-5)
    x *= 0.1

    # clip to [0, 1]
    x += 0.5
    x = np.clip(x, 0, 1)

    # convert to RGB array
    x *= 255
    if K.image_dim_ordering() == 'th':
        x = x.transpose((1, 2, 0))
    x = np.clip(x, 0, 255).astype('uint8')
    return x


def normalize(x):
    """Utility function to normalize a tensor by its L2 norm"""
    return (x + 1e-10) / (K.sqrt(K.mean(K.square(x))) + 1e-10)

### Guided Backprop

In [21]:
def build_guided_model():
    """Function returning modified model.
    
    Changes gradient function for all ReLu activations
    according to Guided Backpropagation.
    """
    if "GuidedBackProp" not in ops._gradient_registry._registry:
        @ops.RegisterGradient("GuidedBackProp")
        def _GuidedBackProp(op, grad):
            dtype = op.inputs[0].dtype
            return grad * tf.cast(grad > 0., dtype) * \
                   tf.cast(op.inputs[0] > 0., dtype)

    #g = tf.get_default_graph()
    g = tf.compat.v1.get_default_graph()
    with g.gradient_override_map({'Relu': 'GuidedBackProp'}):
        new_model = build_model()
    return new_model


def guided_backprop(input_model, images, layer_name):
    """Guided Backpropagation method for visualizing input saliency."""
    input_imgs = input_model.input
    layer_output = input_model.get_layer(layer_name).output
    grads = K.gradients(layer_output, input_imgs)[0]
    backprop_fn = K.function([input_imgs, K.learning_phase()], [grads])
    grads_val = backprop_fn([images, 0])[0]
    return grads_val

### GradCAM

In [22]:
def grad_cam(input_model, image, cls, layer_name):
    """GradCAM method for visualizing input saliency."""
    y_c = input_model.output[0, cls]
    conv_output = input_model.get_layer(layer_name).output
    grads = K.gradients(y_c, conv_output)[0]
    # Normalize if necessary
    # grads = normalize(grads)
    gradient_function = K.function([input_model.input], [conv_output, grads])

    output, grads_val = gradient_function([image])
    output, grads_val = output[0, :], grads_val[0, :, :, :]

    weights = np.mean(grads_val, axis=(0, 1))
    cam = np.dot(output, weights)

    # Process CAM
    cam = cv2.resize(cam, (W, H), cv2.INTER_LINEAR)
    cam = np.maximum(cam, 0)
    cam = cam / cam.max()
    return cam

def grad_cam_batch(input_model, images, classes, layer_name):
    """GradCAM method for visualizing input saliency.
    Same as grad_cam but processes multiple images in one run."""
    loss = tf.gather_nd(input_model.output, np.dstack([range(images.shape[0]), classes])[0])
    layer_output = input_model.get_layer(layer_name).output
    grads = K.gradients(loss, layer_output)[0]
    gradient_fn = K.function([input_model.input, K.learning_phase()], [layer_output, grads])

    conv_output, grads_val = gradient_fn([images, 0])    
    weights = np.mean(grads_val, axis=(1, 2))
    cams = np.einsum('ijkl,il->ijk', conv_output, weights)
    
    # Process CAMs
    new_cams = np.empty((images.shape[0], H, W))
    for i in range(new_cams.shape[0]):
        cam_i = cams[i] - cams[i].mean()
        cam_i = (cam_i + 1e-10) / (np.linalg.norm(cam_i, 2) + 1e-10)
        new_cams[i] = cv2.resize(cam_i, (W, H), cv2.INTER_LINEAR)
        new_cams[i] = np.maximum(new_cams[i], 0)
        new_cams[i] = new_cams[i] / new_cams[i].max()
    
    return new_cams

In [23]:
def compute_saliency(model, guided_model, img_path, layer_name='block5_conv3', cls=-1, visualize=True, save=True):
    """Compute saliency using all three approaches.
        -layer_name: layer to compute gradients;
        -cls: class number to localize (-1 for most probable class).
    """
    preprocessed_input = load_image(img_path)

    predictions = model.predict(preprocessed_input)
    top_n = 5
    top = decode_predictions(predictions, top=top_n)[0]
    classes = np.argsort(predictions[0])[-top_n:][::-1]
    print('Model prediction:')
    for c, p in zip(classes, top):
        print('\t{:15s}\t({})\twith probability {:.3f}'.format(p[1], c, p[2]))
    if cls == -1:
        cls = np.argmax(predictions)
    class_name = decode_predictions(np.eye(1, 1000, cls))[0][0][1]
    print("Explanation for '{}'".format(class_name))
    
    gradcam = grad_cam(model, preprocessed_input, cls, layer_name)
    gb = guided_backprop(guided_model, preprocessed_input, layer_name)
    guided_gradcam = gb * gradcam[..., np.newaxis]

    if save:
        jetcam = cv2.applyColorMap(np.uint8(255 * gradcam), cv2.COLORMAP_JET)
        jetcam = (np.float32(jetcam) + load_image(img_path, preprocess=False)) / 2
        cv2.imwrite('gradcam.jpg', np.uint8(jetcam))
        cv2.imwrite('guided_backprop.jpg', deprocess_image(gb[0]))
        cv2.imwrite('guided_gradcam.jpg', deprocess_image(guided_gradcam[0]))
    
    if visualize:
        plt.figure(figsize=(15, 10))
        plt.subplot(131)
        plt.title('GradCAM')
        plt.axis('off')
        plt.imshow(load_image(img_path, preprocess=False))
        plt.imshow(gradcam, cmap='jet', alpha=0.5)

        plt.subplot(132)
        plt.title('Guided Backprop')
        plt.axis('off')
        plt.imshow(np.flip(deprocess_image(gb[0]), -1))
        
        plt.subplot(133)
        plt.title('Guided GradCAM')
        plt.axis('off')
        plt.imshow(np.flip(deprocess_image(guided_gradcam[0]), -1))
        plt.show()
        
    return gradcam, gb, guided_gradcam

## Computing saliency

In [24]:
model = build_model()
guided_model = build_guided_model()

In [25]:
gradcam, gb, guided_gradcam = compute_saliency(model, guided_model, 'cat_dog.png', layer_name='block5_conv3', 
                                               cls=-1, visualize=True, save=True)
gradcam, gb, guided_gradcam = compute_saliency(model, guided_model, 'cat_dog.png', layer_name='block5_conv3',
                                               cls=282, visualize=True, save=False)

Model prediction:
	bucket         	(463)	with probability 0.005
	cowboy_hat     	(515)	with probability 0.005
	water_bottle   	(898)	with probability 0.004
	tub            	(876)	with probability 0.004
	water_jug      	(899)	with probability 0.004
Explanation for 'bucket'


RuntimeError: tf.gradients is not supported when eager execution is enabled. Use tf.GradientTape instead.

# Generating explanations for many images

Load all images from the folder into one array X

In [ ]:
dir_path = 'dogs_and_cats'
files = os.listdir(dir_path)
N = len(files)

X = np.empty((N, H, W, 3))
for i, file in enumerate(files):
    x = image.load_img(dir_path + file, target_size=(H, W))
    X[i] = image.img_to_array(x)
X = preprocess_input(X)

### Generate and save GradCAM saliency maps

In [ ]:
# Let's set classes for explanations as most probable class for each image.
top = np.argmax(model.predict(X), 1)

In [ ]:
gradcam = np.empty((X.shape[:-1]))
batch_size = 32
for i in range((N + batch_size - 1) // batch_size):
    start = i * batch_size
    end = min((i+1) * batch_size, N)
    gradcam[start:end] = grad_cam_batch(model, X[start:end], top[start:end], 'block5_conv3')
    
gradcam.tofile('gradcam.npy')

i = 63
plt.title(decode_predictions(np.eye(1, 1000, top[i]), 1)[0][0][1])
plt.imshow(np.flip(deprocess_image(X[i]), -1))
plt.imshow(gradcam[i], alpha=0.3, cmap='jet')
plt.show()

### Generate and save Guided Backpropagation saliency maps

In [ ]:
gbp = np.empty((X.shape))
batch_size = 32
for i in range((N + batch_size - 1) // batch_size):
    start = i * batch_size
    end = min((i+1) * batch_size, N)
    gbp[start:end] = guided_backprop(guided_model, X[start:end], 'block5_conv3')
    
gbp.tofile('guided_backprop.npy')

i = 63
plt.title(decode_predictions(np.eye(1, 1000, top[i]), 1)[0][0][1])
plt.imshow(np.flip(deprocess_image(gbp[i]), -1), cmap='jet')
plt.show()

### Generate and save Guided GradCAM

In [ ]:
guided_gradcam = gbp * gradcam[..., np.newaxis]
guided_gradcam.tofile('guided_gradcam.npy')

i = 63
plt.title(decode_predictions(np.eye(1, 1000, top[i]), 1)[0][0][1])
plt.imshow(np.flip(deprocess_image(guided_gradcam[i]), -1), alpha=0.5, cmap='jet')
plt.show()